In [1]:
import hashlib

# ファイルの内容をfileに格納（事前にファイルのダウンロードが必要）
with open('python-3.7.3-embed-win32.zip', 'br') as f:
    file = f.read()
    
# hashlibからmd5のインスタンスを作り、16進数表示のチェックサムを得る
hashlib.md5(file).hexdigest()

'70df01e7b0c1b7042aabb5a3c1e2fbd5'

In [2]:
hashlib.md5('あ'.encode('UTF-8')).hexdigest()

'8c0c3027e3cfc3d644caab3847a505b0'

In [3]:
print(hash(1))
print(hash(2))

1
2


In [4]:
import pickle
print(hashlib.md5(pickle.dumps(1)).hexdigest())
print(hashlib.md5(pickle.dumps(2)).hexdigest())

366da3d0fc5e8ed36f9eac8083a46ae8
76f34d73a1a6753d1243c9ba0afe3457


In [5]:
def to_byte_hash(val):
    byte_data = pickle.dumps(val)
    return hashlib.md5(byte_data).hexdigest()

In [6]:
array_10k = list(range(10000))
to_byte_hash(array_10k)

'390c7c5300a128c476f944872a93ccf4'

In [7]:
array_10k[0] = -1
to_byte_hash(array_10k)

'ab2d4961f8ac559b4e50588ff7992826'

In [8]:
array_10k[0] = 0
to_byte_hash(array_10k)

'390c7c5300a128c476f944872a93ccf4'

In [9]:
def block_to_hash(block_data, nonce):
    # 関数の引数にnonceを連結して、sha256を使ってハッシュ値を計算
    input_str = str(block_data) + str(nonce)
    h = hashlib.sha256(input_str.encode('UTF-8')).hexdigest()
    # 先頭から0がいくつ並んでいるかを数える。
    cnt = 0
    for v in h:
        if v == '0':
            cnt += 1
        else:
            break
    return h, cnt

In [10]:
# 先頭に0が5つ並んだハッシュ値を探す
my_block = 'prev_block_tx0_tx1_tx2'
c = 1
while True:
    hash_val, cnt = block_to_hash(my_block, c)
    if cnt == 5:
        print('{}回目の計算で成功しました'.format(c))
        print(hash_val)
        break
    c += 1

799585回目の計算で成功しました
00000472cdbfc8538016d49f00d7c5ad01cb9e6fc7c733ddbe70ed1e90deb773


In [11]:
p = pow(2, 89) - 1
q = pow(2, 107) - 1
print(p)
print(q)

618970019642690137449562111
162259276829213363391578010288127


In [12]:
n = p*q
n

100433627766186892221372630609062766858404681029709092356097

In [13]:
r = 65537

def find_s(phi_pq, r):
    k = 1
    while True:
        y = 1 + k * phi_pq
        s, d = divmod(y, r)
        if d == 0:
            return k, s
        k += 1

In [14]:
k, s = find_s((p-1)*(q-1), r)
s

15499423397885381203395986760745292550657831765628692176393

In [15]:
rsa_encode = pow
rsa_decode = pow

In [16]:
# 「猫」をUnicodeコードポイントの整数に変換
msg = ord('猫')
msg

29483

In [17]:
secret_msg = rsa_encode(msg, r, n)
secret_msg

82321475112792022243592670346886009619295334480301269316154

In [18]:
decoded_msg = rsa_decode(secret_msg, s, n)
chr(decoded_msg)

'猫'

In [19]:
def block_sort_encode(val):
    res = []
    res.append(val)
    # もとのデータを追加してあるので、forの回数は1つ少ない
    for i in range(len(val) - 1):
        # 1つ前（現在の最新）をもってくる
        temp = res[-1]
        # 先頭の文字を末尾に移動
        res.append(temp[1:] + temp[0])
    res.sort()
    # 元のデータがどこへ行ったか探す
    idx = res.index(val)
    # 末尾の文字をすべて集めたものが結果
    encoded_str = [v[-1] for v in res]
    return ''.join(encoded_str), idx

In [20]:
input_str = '明日の会合は午後が良いですが，良い天気だとお昼ご飯のあとで眠くなってしまうので，午後は午後でも遅い時間が良いです．'

In [21]:
bsorted_str = block_sort_encode(input_str)
bsorted_str

('の良良良遅まと後間す眠昼てでで気なっいい後とのだあく飯う日合後しでのはは，会い午午午明．おい天でがが，も時ごでがす', 43)

In [22]:
def block_sort_decode(val, idx):
    # ソートしたときにそれぞれ何処へいくかを格納する
    char_last_idx = []
    for i, v in enumerate(val):
        char_last_idx.append((v, i))
    # 文字でソートする。同じ文字の場合は、元の順序を保持する。
    char_last_idx.sort()
    # すべて0で初期化
    last_to_front_idx = [0] * len(char_last_idx)
    for i, v in enumerate(char_last_idx):
        # 末尾にあった文字が先頭へ行ったとき何処へ行ったか
        last_to_front_idx[v[1]] = i
    res = val[idx]
    # 最初は、元のデータのあった場所から
    i = last_to_front_idx[idx]
    while i != idx:
        res += val[i]
        #　次に行く場所へ添え字を更新
        i = last_to_front_idx[i]
    # 逆順にして返す
    return res[::-1]

In [23]:
block_sort_decode(*bsorted_str)

'明日の会合は午後が良いですが，良い天気だとお昼ご飯のあとで眠くなってしまうので，午後は午後でも遅い時間が良いです．'

# 練習問題解答

## 10.1

文字列を暗号学的ハッシュ関数で変換するときに、ランダムに生成した小さなデータを付与することがある。これはソルトと呼ばれ、同じ入力に対して、違う出力が出るようにし、わかりやすい文字列を変換した時に情報が漏洩するのを防ぐ役割がある。

関数の名前は、hash_with_saltとしよう。また、saltを引数でとるようにし、渡されなかったときは、内部でランダムに生成しよう。

In [24]:
import random
random.seed(10)

def hash_with_salt(val, salt=None):
    if not salt:
        salt = str(random.randint(1, 100))
    m = hashlib.sha3_512()
    m.update(val.encode('UTF-8') + salt.encode('UTF-8'))
    return m.hexdigest(), salt

In [25]:
hash_with_salt('password')

('f129003122900957bc0b3e324bbb9cf2970c35555de9ab7accc2be7374ce4ea7f22f877c0a9bb994552e4daff00214a9aa6026807439003a40118f9d694f2949',
 '74')

文字列をバイト列に変換するために、encodeメソッドを使った。ハッシュ関数への入力に小さなデータを加えれば、元の文字列がpasswordであることは推測されにくくなる。なぜなら、単純にsha3_512アルゴリズムを使った場合とは全く違う文字列になるからだ。

In [26]:
hashlib.sha3_512(b'password').hexdigest()

'e9a75486736a550af4fea861e2378305c4a555a05094dee1dca2f68afea49cc3a50e8de6ea131ea521311f4d6fb054a146e8282f8e35ff2e6368c1a62e909716'

saltがわかっていれば、受け取った文字列と暗号化した文字列を照合できる。

In [27]:
hash_with_salt('password', '74')

('f129003122900957bc0b3e324bbb9cf2970c35555de9ab7accc2be7374ce4ea7f22f877c0a9bb994552e4daff00214a9aa6026807439003a40118f9d694f2949',
 '74')

## 10.2

9章の練習問題で見付けた素数から2つを持ってこよう。これらをｐとｑとして、掛け合わせた数をnとする。これは公開鍵の一部となるが、大きな数の素因数分解には時間がかかるので、nからｐとｑを推測することは現代の計算機では難しい。

In [28]:
p = 13029484142325529160278057675859771418199793915415887667143262604822998091867456519520695610374545430551058468053152467257405794784202459324043791117505269283282033102238957121905893454537518737843627
# こちらを使っても良い。p = 2**521 - 1
q = 88301801407902026726739202138197965620005436413133983858109294983959431890532621456825116199688536186064925748393033767927900577431907264510871051225548778689126932900125905788253448444634254212249591
# q = 2**607 -1
n = p * q

r = 65537として、秘密鍵sを計算しよう。

In [29]:
k, s = find_s((p-1)*(q-1), r)

わかりやすいように名前を付けておこう。

In [30]:
public_key = r
private_key = s

あとはこの章で作った組み込み関数powの別名で暗号化と復号化ができるが、powをそのまま使っても構わない。また、折角なので文字列を暗号化してみよう。

In [31]:
top_secret = 'とんでもない秘密'

In [32]:
int(top_secret.encode('UTF-8').hex(), 16)

5578442738491896392425476467725517650170222265267332165510

In [33]:
#　機密情報を整数に変換する
top_secret_int = int(top_secret.encode('UTF-8').hex(), 16)
top_secret_int

5578442738491896392425476467725517650170222265267332165510

In [34]:
# 暗号化する。
crypto_data = pow(top_secret_int, public_key, n)
crypto_data

1052983848199668843728459043526597292807303529046302034682346063883032627421508627032086221876808727163856446581770021494470979583782194461247570709679818184711552945620729052807414218209978482442376731211199402646372412318947712356396703420765398057776596808275134648818061897098517310132292631308276045391792748749449415336216662502846487852602584423235134206457513329587038135791916661154965541380

In [35]:
# 送られて来た暗号を復号化する
received_data = pow(crypto_data, private_key, n)

In [36]:
received_data

5578442738491896392425476467725517650170222265267332165510

In [37]:
# 整数値をもとの文字列に戻す
bytes.fromhex(format(received_data, 'x')).decode('UTF-8')

'とんでもない秘密'

## 10.3

httpsからはじまるURLをもったサイトであれば、なんでもよい。たとえば、https://www.python.org　などブラウザで開き、アドレスバーの横（通常左側）にある南京錠のマークをクリックすると、HTTPS通信のために利用されている公開鍵暗号基盤の証明書の情報などが参照できる。ハッシュのアルゴリズムにSHA256が使われていることなどが確認できるだろう。公開鍵にはRSA暗号が使われていることが多いが、一部サイトでは楕円曲線暗号と呼ばれるアルゴリズムが使われていることもある。このように、現代社会はアルゴリズムに支えられている。また、この分野は日進月歩であるため、基礎的な知識を学習したあとも、常に新しい情報を獲得し理解し続ける姿勢が重要となる。